# Demo Wine avec MLflow

## Setup MLflow
Par defaut suppose que le repository est le répertoire local mlruns

Pour travailler en coopération, on va désigner le serveur 
soit en ligne de commande avec
```
export MLFLOW_TRACKING_URI=http://localhost:5000
```
soit par programme

In [3]:
import mlflow
server_uri = "http://localhost:5000"
mlflow.set_tracking_uri(server_uri)

## Création d'une expérimentation

L'expérimentation peut être crée puis sélectionnée en ligne de commande par
```
mlflow experiments create --experiment-name ac2  --artifact-location /opt/mlflow/mlruns/
export MLFLOW_EXPERIMENT_ID=1
```
ou par programme (voir ci-après)

In [4]:
mlflow.set_experiment("wine7")

## Entraînement

In [5]:
import warnings
import logging
logging.basicConfig(level=logging.WARN)
logger = logging.getLogger(__name__)

import numpy as np
warnings.filterwarnings("ignore")
np.random.seed(40)

import mlflow.sklearn

In [6]:
import pandas as pd 

# read data from file
df = pd.read_csv("wine-quality.csv")

In [7]:
from sklearn.model_selection import train_test_split
# Split the data into training and test sets. (0.75, 0.25) split.
train, test = train_test_split(df)

# The predicted column is "quality" which is a scalar from [3, 9]
train_x = train.drop(["quality"], axis=1)
test_x = test.drop(["quality"], axis=1)
train_y = train[["quality"]]
test_y = test[["quality"]]

# make it easy to pass datasets
datasets = { 'train_x': train_x, 'train_y': train_y, 'test_x': test_x, 'test_y': test_y }

In [8]:
shapes = [ "%s : %s" % (name, dataset.shape) for (name,dataset) in datasets.items() ]
print(shapes)

['train_x : (3673, 11)', 'train_y : (3673, 1)', 'test_x : (1225, 11)', 'test_y : (1225, 1)']


In [9]:
def eval_parameters(in_alpha, in_l1_ratio):
    # Set default values if no alpha is provided
    alpha = float(in_alpha) if not in_alpha is None else 0.5
    # Set default values if no l1_ratio is provided
    l1_ratio = float(in_l1_ratio) if not in_l1_ratio is None else 0.5
    return alpha, l1_ratio

In [10]:
def eval_metrics(actual, predicted):
    from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
    rmse = np.sqrt(mean_squared_error(actual, predicted))
    mae = mean_absolute_error(actual, predicted)
    r2 = r2_score(actual, predicted)
    return rmse, mae, r2


In [11]:
from sklearn.linear_model import lasso_path, enet_path
import matplotlib.pyplot as plt
from itertools import cycle
def plot_enet_descent_path(tmpdir, X, y, l1_ratio):
    # Compute paths
    eps = 5e-3  # the smaller it is the longer is the path

    # Reference the global image variable
    global image
    
    print("Computing regularization path using ElasticNet.")
    alphas_enet, coefs_enet, _ = enet_path(X, y, eps=eps, l1_ratio=l1_ratio, fit_intercept=False)

    # Display results
    fig = plt.figure(1)
    ax = plt.gca()

    colors = cycle(['b', 'r', 'g', 'c', 'k'])
    neg_log_alphas_enet = -np.log10(alphas_enet)
    for coef_e, c in zip(coefs_enet, colors):
        l1 = plt.plot(neg_log_alphas_enet, coef_e, linestyle='--', c=c)

    plt.xlabel('-Log(alpha)')
    plt.ylabel('coefficients')
    title = 'ElasticNet Path by alpha for l1_ratio = ' + str(l1_ratio)
    plt.title(title)
    plt.axis('tight')

    # Display images
    image = fig
    
    # Save figure
    fig.savefig(os.path.join(tempdir, "ElasticNet-paths.png"))

    # Close plot
    plt.close(fig)

    # Return images
    return image    


In [12]:
import os
def output_enet_coefs(tempdir, columns, lr):
    coef_file_name = os.path.join(tempdir, "coefs.txt")
    with open(coef_file_name, "w") as f: 
        f.write("Coefs:\n")         
        [ f.write("\t %s: %s\n" % (name, coef)) for (name, coef) in zip(columns, lr.coef_) ]
        f.write("\t intercept: %s\n" % lr.intercept_) 

In [13]:
def plot_enet_feature_importance(tempdir, columns, coefs):
    # Reference the global image variable
    global image
    
    # Display results
    fig = plt.figure(1)
    ax = plt.gca()
    
    feature_importance = pd.Series(index = columns, data = np.abs(coefs))
    n_selected_features = (feature_importance>0).sum()
    print('{0:d} features, reduction of {1:2.2f}%'.format(
        n_selected_features,(1-n_selected_features/len(feature_importance))*100))
    feature_importance.sort_values().tail(30).plot(kind = 'bar', figsize = (20,12));
    
    # Display images
    image = fig
    
    # Save figure
    fig.savefig(os.path.join(tempdir, "feature-importance.png"))

    # Close plot
    plt.close(fig)

    # Return images
    return image    

In [14]:
import tempfile
def train_elasticnet(in_alpha, in_l1_ratio, trial=None):
    from sklearn.linear_model import ElasticNet

    alpha, l1_ratio = eval_parameters(in_alpha, in_l1_ratio)
    print("Parameters (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
    
    run_name = "en_%f_%f" % (alpha,l1_ratio)
    with mlflow.start_run() as run:
        # train with ElasticNet
        lr = ElasticNet(alpha=alpha, l1_ratio=l1_ratio, random_state=42)
        lr.fit(train_x, train_y)

        # Evaluate Metrics
        predicted_qualities = lr.predict(test_x)
        (rmse, mae, r2) = eval_metrics(test_y, predicted_qualities)

        # Print out metrics
        print("Elasticnet model (alpha=%f, l1_ratio=%f):" % (alpha, l1_ratio))
        print("  RMSE: %s" % rmse)
        print("  MAE: %s" % mae)
        print("  R2: %s" % r2)

        # Log parameter, metrics, and model to MLflow
        mlflow.log_param("alpha", alpha)
        mlflow.log_param("l1_ratio", l1_ratio)
        mlflow.log_metric("rmse", rmse)
        mlflow.log_metric("mae", mae)
        mlflow.log_metric("r2", r2)
        mlflow.set_tag("algo", "ElastiNet")
        if trial != None:
             mlflow.set_tag("trial", trial)
        
        #run_id = run.info.run_id
        # store info       
        with tempfile.TemporaryDirectory() as tmpdirname:  
            output_enet_coefs(tmpdirname, train_x.columns, lr) 

            # plots
            plot_enet_feature_importance(tmpdirname, train_x.columns, lr.coef_)
            # Call plot_enet_descent_path
            #image = plot_enet_descent_path(tmpdirname, train_x, train_y, l1_ratio)

            # Log artifacts (output files)
            mlflow.log_artifacts(tmpdirname, artifact_path="artifacts")


        # store model
        mlflow.sklearn.log_model(lr, "model")
        
        return rmse

In [15]:
train_elasticnet(0.5, 0.5) # alpha 0.5, L1 0.5

Parameters (alpha=0.500000, l1_ratio=0.500000):
Elasticnet model (alpha=0.500000, l1_ratio=0.500000):
  RMSE: 0.82224284975954
  MAE: 0.6278761410160693
  R2: 0.12678721972772689
3 features, reduction of 72.73%


0.82224284975954

In [16]:
train_elasticnet(0.5, 0.4)

Parameters (alpha=0.500000, l1_ratio=0.400000):
Elasticnet model (alpha=0.500000, l1_ratio=0.400000):
  RMSE: 0.8134651568415164
  MAE: 0.6249674191990106
  R2: 0.1453313314484228
3 features, reduction of 72.73%


0.8134651568415164

## Auto tuning

In [17]:
!pip install optuna

In [18]:
import optuna
from datetime import datetime
optuna.logging.set_verbosity(optuna.logging.INFO)
ts = datetime.now().strftime('%Y%m%dT%H%M%S')
# Define an objective function to be minimized.
def objective(trial):

    suggested_alpha = trial.suggest_uniform('alpha', 0.1, 0.8)
    suggested_l1_ratio = trial.suggest_uniform('l1_ratio', 0.1, 0.8)
        
    error = train_elasticnet(suggested_alpha, suggested_l1_ratio, trial="%s_%s" % (ts,trial.trial_id))
    
    return error  # A objective value linked with the Trial object.

study = optuna.create_study()  # Create a new study.
study.optimize(objective, n_trials=40)  #100   # Invoke optimization of the objective function.

[W 2019-11-21 16:45:48,704] The use of `Trial.trial_id` is deprecated. Please use `Trial.number` instead.


Parameters (alpha=0.284014, l1_ratio=0.286575):
Elasticnet model (alpha=0.284014, l1_ratio=0.286575):
  RMSE: 0.7926654218357168
  MAE: 0.6186358180524036
  R2: 0.1884791183198683
4 features, reduction of 63.64%


[I 2019-11-21 16:45:49,156] Finished trial#0 resulted in value: 0.7926654218357168. Current best value is 0.7926654218357168 with parameters: {'alpha': 0.28401433914824936, 'l1_ratio': 0.28657524954696023}.
[W 2019-11-21 16:45:49,159] The use of `Trial.trial_id` is deprecated. Please use `Trial.number` instead.


Parameters (alpha=0.169605, l1_ratio=0.568143):
Elasticnet model (alpha=0.169605, l1_ratio=0.568143):
  RMSE: 0.791071871143961
  MAE: 0.6183016651201375
  R2: 0.19173875276830044
4 features, reduction of 63.64%


[I 2019-11-21 16:45:49,603] Finished trial#1 resulted in value: 0.791071871143961. Current best value is 0.791071871143961 with parameters: {'alpha': 0.16960522906120284, 'l1_ratio': 0.5681434297161756}.
[W 2019-11-21 16:45:49,606] The use of `Trial.trial_id` is deprecated. Please use `Trial.number` instead.


...

[I 2019-11-21 16:46:07,010] Finished trial#37 resulted in value: 0.7860996784474846. Current best value is 0.7817034255185242 with parameters: {'alpha': 0.14692853335359343, 'l1_ratio': 0.10024493519776644}.
[W 2019-11-21 16:46:07,025] The use of `Trial.trial_id` is deprecated. Please use `Trial.number` instead.


Parameters (alpha=0.199078, l1_ratio=0.197892):
Elasticnet model (alpha=0.199078, l1_ratio=0.197892):
  RMSE: 0.7858128367650332
  MAE: 0.6154772025594029
  R2: 0.20244964929296005
5 features, reduction of 54.55%


[I 2019-11-21 16:46:07,515] Finished trial#38 resulted in value: 0.7858128367650332. Current best value is 0.7817034255185242 with parameters: {'alpha': 0.14692853335359343, 'l1_ratio': 0.10024493519776644}.
[W 2019-11-21 16:46:07,531] The use of `Trial.trial_id` is deprecated. Please use `Trial.number` instead.


Parameters (alpha=0.132520, l1_ratio=0.549703):
Elasticnet model (alpha=0.132520, l1_ratio=0.549703):
  RMSE: 0.7872905485324166
  MAE: 0.6165212332938566
  R2: 0.19944726091908938
4 features, reduction of 63.64%


[I 2019-11-21 16:46:08,032] Finished trial#39 resulted in value: 0.7872905485324166. Current best value is 0.7817034255185242 with parameters: {'alpha': 0.14692853335359343, 'l1_ratio': 0.10024493519776644}.


In [19]:
study.best_params

{'alpha': 0.14692853335359343, 'l1_ratio': 0.10024493519776644}

In [20]:
study.best_trial

FrozenTrial(number=34, state=TrialState.COMPLETE, value=0.7817034255185242, datetime_start=datetime.datetime(2019, 11, 21, 16, 46, 4, 985854), datetime_complete=datetime.datetime(2019, 11, 21, 16, 46, 5, 479785), params={'alpha': 0.14692853335359343, 'l1_ratio': 0.10024493519776644}, distributions={'alpha': UniformDistribution(high=0.8, low=0.1), 'l1_ratio': UniformDistribution(high=0.8, low=0.1)}, user_attrs={}, system_attrs={'_number': 34}, intermediate_values={}, trial_id=34)

## Identification par tag du best model

In [21]:
trial_id = study.best_trial.number
print("trial_id: %s" % trial_id)
tag = "%s_%s" % (ts, trial_id)
print("tag: %s" % tag)

trial_id: 34
tag: 20191121T164548_34


In [22]:
from mlflow.entities import ViewType
query = "tags.trial = '%s'" % tag
runs = mlflow.search_runs(filter_string=query, run_view_type=ViewType.ACTIVE_ONLY)
runs.head(10)

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.mae,metrics.rmse,metrics.r2,params.alpha,params.l1_ratio,tags.mlflow.source.name,tags.mlflow.source.type,tags.algo,tags.trial,tags.mlflow.user
0,5f9e6c70a30242569409540f4e3d7494,9,FINISHED,/opt/mlflow/mlruns/9/5f9e6c70a30242569409540f4...,2019-11-21 16:46:05.002000+00:00,2019-11-21 16:46:05.344000+00:00,0.612994,0.781703,0.210769,0.14692853335359343,0.10024493519776644,/opt/miniconda/envs/demo/lib/python3.7/site-pa...,LOCAL,ElastiNet,20191121T164548_34,ubuntu


In [23]:
best_model_id = runs['run_id'][0]
best_model_uri = runs['artifact_uri'][0]
print("best model - id: %s - uri: %s" % (best_model_id, best_model_uri) )

best model - id: 5f9e6c70a30242569409540f4e3d7494 - uri: /opt/mlflow/mlruns/9/5f9e6c70a30242569409540f4e3d7494/artifacts


In [24]:
from  mlflow.tracking import MlflowClient
mlflow_client = MlflowClient()
mlflow_client.set_tag(best_model_id, "best_model", "true")

In [25]:
runs = mlflow.search_runs(filter_string=query, run_view_type=ViewType.ACTIVE_ONLY)
runs.head(10)

,run_id,experiment_id,status,artifact_uri,start_time,end_time,metrics.mae,metrics.rmse,metrics.r2,params.alpha,params.l1_ratio,tags.mlflow.source.name,tags.best_model,tags.mlflow.source.type,tags.algo,tags.trial,tags.mlflow.user
0,5f9e6c70a30242569409540f4e3d7494,9,FINISHED,/opt/mlflow/mlruns/9/5f9e6c70a30242569409540f4...,2019-11-21 16:46:05.002000+00:00,2019-11-21 16:46:05.344000+00:00,0.612994,0.781703,0.210769,0.14692853335359343,0.10024493519776644,/opt/miniconda/envs/demo/lib/python3.7/site-pa...,true,LOCAL,ElastiNet,20191121T164548_34,ubuntu
